### 한빛출판사 새로나온책

In [83]:
import requests
from urllib.parse import quote
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm
from datetime import datetime 

In [84]:
# https://www.hanbit.co.kr/store/books/new_book_list.html?page=1&brand=&cate1=&cate2=&searchKey=&keyWord=
base_url = 'https://www.hanbit.co.kr/academy/books/new_book_list.html?page='
sub_url = 'https://www.hanbit.co.kr/academy/books/book_view.html?p_code='
other_url = '&cate_cd=&srt=&searchKey=&keyWord='
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'}
addr_head = 'https://www.hanbit.co.kr' 

In [85]:
url = f'{base_url}1{other_url}'
result = requests.get(url, headers=header)
soup = BeautifulSoup(result.text, 'html.parser')


In [86]:
lis = soup.select('.sub_book_list_area > li')
len(lis)

20

In [87]:
blist = lis[0]
price = int(blist.select_one('.price').get_text().strip('원').replace(',', ''))
title = blist.select_one('.book_tit > a').get_text().strip()
author = blist.select_one('.book_writer').get_text().strip()
img = addr_head + blist.select_one('.view_box_block > img')['src']


In [88]:
lines = []
for page in tqdm(range(1, 24)):
    url = f'{base_url}{page}{other_url}'
    result = requests.get(url, headers=header)
    soup = BeautifulSoup(result.text, 'html.parser')

    lis = soup.select('.sub_book_list_area > li')
    for li in lis:
        price = int(li.select_one('.price').get_text().strip('원').replace(',', ''))
        title = li.select_one('.book_tit > a').get_text().strip()
        author = li.select_one('.book_writer').get_text().strip()
        img = addr_head + li.select_one('.view_box_block > img')['src']
        lines.append({'title': title, 'author': author, 'price': price, 'img': img})

df = pd.DataFrame(lines)
df.head()



100%|██████████| 23/23 [00:18<00:00,  1.28it/s]


,title,author,price,img
0,처음 배우는 매트랩,방성완,30000,https://www.hanbit.co.kr/data/books/B719243717...
1,SAS와 메타분석을 활용한 한번에 적용하는 의학보건통계,히든그레이스 데이터분석팀,34000,https://www.hanbit.co.kr/data/books/B911113579...
2,정보교과교육론(3판),"최현종 , 전용주",30000,https://www.hanbit.co.kr/data/books/B698322643...
3,"IT CookBook, 기초전자실험 with PSpice(3판)",홍순관,22000,https://www.hanbit.co.kr/data/books/B543265656...
4,경제수학 강의(3판),김성현,23000,https://www.hanbit.co.kr/data/books/B346928828...


In [89]:
now = datetime.now()
cur_day = now.strftime('%Y%m%d')

df.to_csv(f'data/{cur_day}_한빛아카데미새책.csv', index=False)

- sub page에서 가져오기

In [118]:
lines = []
for page in tqdm(range(1, 2)):
    url = f'{base_url}{page}{other_url}'
    result = requests.get(url, headers=header)
    soup = BeautifulSoup(result.text, 'html.parser')

    lis = soup.select('.sub_book_list_area > li')
    for li in lis:
        price = int(li.select_one('.price').get_text().strip('원').replace(',', ''))
        # title = li.select_one('.book_tit > a').get_text().strip()
        # author = li.select_one('.book_writer').get_text().strip()
        # img = addr_head + li.select_one('.view_box_block > img')['src']
        sub_url = addr_head + li.select_one('.pop_quick_menu > a')['href']
        sub_res = requests.get(sub_url, headers=header)
        sub_soup = BeautifulSoup(sub_res.text, 'html.parser')
        title = sub_soup.select_one('.store_product_info_box > h3').get_text().strip()
        li_list = sub_soup.select('.info_list > li')
        author = li_list[0].find('span').get_text().replace(' , ', ', ')
        
        if len(li_list) == 5:
            translator = '-'
        else:
            translator = li_list[1].find('span').get_text().replace(' , ', ', ')
        

        # 출간일
        rel_date = li_list[-4].find('span').get_text().strip()
        # 페이지
        book_page = li_list[-3].find('span').get_text().strip()[:-2]
        # 가격정보
        price = sub_soup.select('.payment_box.curr > p > span')
        print(price)

        # lines.append({'title': title, 'author': author, 'price': price, 'img': img})


  0%|          | 0/1 [00:00<?, ?it/s]

[<span class="pbl">정가 : </span>, <span class="pbr"><del>30,000원</del></span>, <span class="pbl"><strong>판매가 : </strong></span>, <span class="pbr"><strong>30,000</strong>원<span>(0% off)</span></span>, <span class="pbl">마일리지 : </span>, <span class="pbr">900점 (3%)</span>]
[<span class="pbl">정가 : </span>, <span class="pbr"><del>34,000원</del></span>, <span class="pbl"><strong>판매가 : </strong></span>, <span class="pbr"><strong>34,000</strong>원<span>(0% off)</span></span>, <span class="pbl">마일리지 : </span>, <span class="pbr">1,020점 (3%)</span>]
[<span class="pbl">정가 : </span>, <span class="pbr"><del>30,000원</del></span>, <span class="pbl"><strong>판매가 : </strong></span>, <span class="pbr"><strong>30,000</strong>원<span>(0% off)</span></span>, <span class="pbl">마일리지 : </span>, <span class="pbr">900점 (3%)</span>]
[<span class="pbl">정가 : </span>, <span class="pbr"><del>22,000원</del></span>, <span class="pbl"><strong>판매가 : </strong></span>, <span class="pbr"><strong>22,000</strong>원<span>(0% off)</sp